In [98]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
import os
from web3 import Web3
from pprint import pprint
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account

In [99]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [100]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json'
   
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys


In [101]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc", "btc-test"}
print(coins)
numderive = 3


{'eth', 'btc', 'btc-test'}


In [102]:
key = {}
for coin in coins:
    key[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [103]:
eth_PrivateKey = key["eth"][0]['privkey']
btc_PrivateKey = key["btc-test"][0]['privkey']

print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0xbdcd89c18c95bd64ca9a37ee12432d27913b921d1f9434a542e452f7ee05fcf1"
"cUxqurDa1GvAtWjGH7MW7TMoRNoTg9bkghyvCKEZjWoR2UUSc7fY"


In [104]:
print(json.dumps(key, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "17ZUhCWLYpHZm79nQFTFQLaFwvxSfd1EBx",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L4jdVndebH2k3JykQesK8bxkYX2FAEQGTqhU9c6gFvQLemaz1yh4",
            "pubkey": "036e69cb51fca260aae092cb4d96fb180ad47ceb5095d97d72fd334ef84618b21b",
            "pubkeyhash": "47f52eb8006d26ea37b22e82cdbd999b819e6515",
            "xprv": "xprvA2osriaVTKgz8Yo4SqQ9mFE2i5dvLUiUT2SbEa2jPeLzGU85E5XL9J8G8qi3Wkx717ad5pXyPy25FRQxzwmdGfkZfpYr5Z2sqwdsRsGYUom",
            "xpub": "xpub6FoEGE7PHhFHM2sXYrwA8PAmG7UQjwSKpFNC2xSLwysy9GTDmcqah6Sjz7pREtkG7DzQ7KQaXUd1CRMxC7vfyDNnCA1bVHKmJ7hcaMRS7Am"
        },
        {
            "address": "1GCD1qRU853a2ngBaES39KAyTQr99vxXvK",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L4W9amSCpK57EtaTor9oqYFNT62n28BPtFnyGkpF8Vg2JEy5mcF2",
            "pubkey": "027e06d900cf36e2aecf4c33eb2987ebcb15049c4f31017578ce01e0b2809752d2",
            "pubkeyhash":

In [105]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.

def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)






In [106]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [107]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [108]:
## BTC TEST Transactions

In [109]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cUxqurDa1GvAtWjGH7MW7TMoRNoTg9bkghyvCKEZjWoR2UUSc7fY


In [110]:
create_tx(BTCTEST,btc_acc,"mv791vkCdMEkV3D95u6WY6dKVC3fNffqkZ", 0.0001)

'{"unspents":[{"amount":121523,"confirmations":18,"script":"76a914a008da99f38bb77c78ce7164c8801acd7873f88b88ac","txid":"6a9bb1d9a137831ac9134164845c304c44213cf8b2df5555ec3653021775639d","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mv791vkCdMEkV3D95u6WY6dKVC3fNffqkZ",10000],["mv791vkCdMEkV3D95u6WY6dKVC3fNffqkZ",88471]]}'

In [111]:
# Send BTC Transaction
send_txn(BTCTEST,btc_acc,"mtK73sNPY9CKuzVvpv4W1969AD1UmGGfsX", 0.00001)

01000000019d637517025336ec5555dfb2f83c21444c305c84644113c91a8337a1d9b19b6a010000006a47304402203fe4371d8750d3355f87d42602696c230e9f64759c627204a22c5f6699694d2002203dee18c8b8ed3394096fcc3b8fbeeb2201e957a786935c512a5086f8122b1a8e012103ae7763f48afea82109fde354439d984fd284b45252f2e35aae9bd1100f9d441fffffffff02e8030000000000001976a9148c5bf3cb05705fe9512d23b229c4253e82ba817988acbf7c0100000000001976a914a008da99f38bb77c78ce7164c8801acd7873f88b88ac00000000


In [ ]:
# ETH Test Transactions

In [113]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [114]:
# Connecting to HTTP with address
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/762e30385c647fa440ad0eeb4637b79bb39ae350d115334f37e846739533b6e8"))



In [115]:
# Connection Test
w3.isConnected()

True

In [116]:
#Mic-check 

In [145]:
w3.eth.getBalance("0xF5D5964a196EfDF5076D2EbE9222D515dafd1B8a")

904625697166532776746648320380374280103671755200316906558262375061821325312

In [147]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }



In [148]:
def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()




In [149]:
## Confirmation of Transaction

In [151]:
w3.eth.getBalance("0xF5D5964a196EfDF5076D2EbE9222D515dafd1B8a")

#0xF5D5964a196EfDF5076D2EbE9222D515dafd1B8a
#0x93dB98E9aBaFFf52ba1bCEB835d6c834b546440B

844118827091047554464641041613674348052465730571